### Setting up connection to Celonis EMS

In [1]:
import pandas as pd

from pycelonis import get_celonis
from pycelonis import pql

ModuleNotFoundError: No module named 'pycelonis'

In [2]:
celonis = get_celonis()

NameError: name 'get_celonis' is not defined

In [ ]:
datamodel = celonis.datamodels.find('XXX') # Insert datamodel ID / name

In [ ]:
# Functions are calclulating the values in 'PTI' and 'Requirement' columns 

def update_value_PTI(text: str):
    for classification in list_of_classifications:
        if classification in text:
            return classification
        

def update_value_Requirement(text: str):
    for Kurzbezeichnung in list_of_Kurzbezeichnung:
        if Kurzbezeichnung in text:
            return Kurzbezeichnung

# Preparing the string which will be used as PQL filter later
def prepare_string_statement(seznam: list):
    string = 'Filter '
    for classification in seznam:
        string = string + f"TXZ01.TXZ01 LIKE '%{classification}%' OR " 
    
    if string[-1] == ' ':
        string = string[:-4]
    string = string + ' ;'
    return string    


### Creation of initials dataframes

In [ ]:
query_2 = pql.PQL()
query_2 += pql.PQLColumn(query = "Classification_abbreviations.Klassifizierung", name = "Klassifizierung")

df_2 = datamodel.get_data_frame(query_2)
list_of_classifications = df_2["Klassifizierung"].to_list()

In [ ]:
query = pql.PQL()
query += pql.PQLColumn(query = "TXZ01.TXZ01", name = "TXZ01")
query += pql.PQLColumn(query = "TXZ01.EBELN", name = "EBELN")
query += pql.PQLColumn(query = "TXZ01.EBELP", name = "EBELP")

# query += pql.PQLFilter("Filter TXZ01.TXZ01 LIKE '%N23%';")

query += pql.PQLFilter(prepare_string_statement(list_of_classifications))
df_filtered = datamodel.get_data_frame(query)

df_filtered.drop_duplicates(inplace = True)

In [ ]:
query_3 = pql.PQL()
query_3 += pql.PQLColumn(query = "Classification_types_of_business.Kurzbezeichnung", name = "Kurzbezeichnung")
query_3 += pql.PQLColumn(query = "Classification_types_of_business.Baseline", name = "Baseline")

df_3 = datamodel.get_data_frame(query_3)
list_of_Kurzbezeichnung = df_3["Kurzbezeichnung"].to_list()

### Filtering of dataframes

In [ ]:
df_filtered['PTI'] = df_filtered['TXZ01'].apply(update_value_PTI)
df_filtered['Requirement'] = df_filtered['TXZ01'].apply(update_value_Requirement)

# Setting of NULL values in column 'Requirement' to 'N/A'
df_filtered = df_filtered.assign(Requirement=df_filtered['Requirement'].mask(df_filtered['Requirement'].isna(), 'N/A'))

### Joining the dataframes in order to get 'TYPE_OF_BUSINESS' and create final df

In [ ]:
df_3 = df_3.rename(columns={"Kurzbezeichnung": "Requirement", "Baseline": "TYPE_OF_BUSINESS"})

df_joined = pd.merge(df_filtered, df_3, on='Requirement', how = 'left')

In [ ]:
# Setting of NULL values in column 'TYPE_OF_BUSINESS' to 'Projektgeschäft'
df_final = df_joined.assign(TYPE_OF_BUSINESS=df_joined['TYPE_OF_BUSINESS'].mask(df_joined['TYPE_OF_BUSINESS'].isna(), 'Projektgeschäft'))

### Pushing the data into EMS

In [ ]:
data_pool = celonis.pools.find("XXX") # insert datapool ID

In [ ]:
# push to EMS and execute the datajob transform the table into both development and production environment

data_pool.create_table(table_name="fill_PTI_AND_TYPE_OF_BUSINESS_PER_EBELN_PYTHON",
                   df_or_path=df_final,
                   if_exists="drop")

datajob = data_pool.data_jobs.find("XXX") # Insert datapool ID
datajob.execute()

print("Data successfully pushed into EMS.")